# Minesweeper
---
_**Module**: HSLU - Artificial Intelligence - Search & Optimization (AISO)_  
_**Source**: Slides "Constraint programming 1 - Modelling with OR-Tools"_  
_**Author**: Adrian Kauz_

>The player is presented with a grid of squares. Some randomly selected squares, unknown to the player, contain mines 💣. Other squares may display numbers indicating the count of mines present in the immediate neighborhood of the squares. Find all mines!

![Minesweeper](images/Minesweeper.png)

## Imports

In [1]:
from ortools.constraint_solver import pywrapcp
import numpy as np

## Helper functions

In [2]:
def printSolution(currentSolution):
    print("Possible solution:\n")
    
    for row in range(len(currentSolution)):
        currentRow = ""
        for column in range(len(currentSolution[row])):
            if (currentSolution[row][column][1].Value() == -1):
                currentRow += "●  "
            elif (currentSolution[row][column][1].Value() == 0):
                if (currentSolution[row][column][0] == 0):
                    currentRow += "0  "
                else:
                    currentRow += "∙  "
            else:
                currentRow += "{}  ".format(currentSolution[row][column][0])
        
        print(currentRow)
    
    print("")

## Puzzle Examples

In [4]:
_ = -1
puzzle = {}

# Unknown Source
puzzle["1"] = [
    [1, _, _, _, 2, _, _, 1],
    [_, 1, 2, _, 3, _, _, 1],
    [_, 2, _, 1, _, _, _, 0],
    [_, 2, 1, _, _, 2, 3, 1],
    [_, _, _, 2, _, _, _, _],
    [1, _, _, _, 4, 3, _, _],
    [_, 1, _, _, 4, _, 3, _],
    [1, _, _, 2, _, 2, _, 1]
]

# From Slide
puzzle["2"] = [
    [2, _, _, _, 1],
    [2, _, 4, 3, _],
    [_, 2, _, 1, _],
    [_, 1, _, 3, _],
    [1, _, _, _, _]
]

# From https://www.gmpuzzles.com/blog/minesweeper-rules-and-info/
puzzle["3"] = [
    [_, _, _, _, _, _, _, _, _, _],
    [_, 3, 4, 5, _, _, 5, 3, 1, _],
    [_, 2, _, _, _, _, _, _, 2, _],
    [_, 1, _, 2, 2, 2, 2, _, 4, _],
    [_, _, _, 2, _, _, 2, _, _, _],
    [_, _, _, 2, _, _, 2, _, _, _],
    [_, 4, _, 2, 2, 2, 2, _, 1, _],
    [_, 2, _, _, _, _, _, _, 2, _],
    [_, 1, 2, 4, _, _, 1, 2, 4, _],
    [_, _, _, _, _, _, _, _, _, _]
]

# From http://www.cross-plus-a.com/puzzles.htm#Minesweeper
puzzle["4"] = [
    [_, 1, _, 2, 2, 2, _, 1, _, 1],
    [_, 3, _, _, _, _, 2, _, 1, _],
    [_, 2, _, _, 3, 3, _, _, _, 2],
    [_, _, _, _, _, _, _, _, _, 1],
    [_, 2, _, 3, 2, _, _, 2, _, _],
    [_, _, 4, _, _, 1, 2, _, 2, _],
    [1, _, _, _, _, _, _, _, _, _],
    [2, _, _, _, 3, 4, _, _, 2, _],
    [_, 3, _, 1, _, _, _, _, 4, _],
    [2, _, 1, _, 2, 3, 4, _, 4, _]
]

# From http://www.cross-plus-a.com/puzzles.htm by Serkan Yürekli
puzzle["5"] = [
    [_, 1, 1, 1, _, _, 1, 1, 2, _, _, _],
    [_, _, 2, _, _, _, 1, _, _, _, 3, 2],
    [1, _, _, _, _, _, _, _, _, 1, 1, _],
    [1, _, _, 1, 2, 2, 1, _, _, _, _, _],
    [2, 3, _, _, _, _, _, _, 1, _, 1, 1],
    [_, _, _, _, 1, _, _, 1, 2, _, 3, 2],
    [2, 3, _, 1, 1, _, _, 1, _, _, _, _],
    [1, 2, _, 3, _, _, _, _, _, _, 3, 2],
    [_, _, _, _, _, 2, 2, 1, 1, _, _, 1],
    [_, 2, 2, _, _, _, _, _, _, _, _, 3],
    [2, 3, _, _, _, 1, _, _, _, 2, _, _],
    [_, _, _, 2, 1, 1, _, _, 2, 1, 2, _],
]

## Solution

In [5]:
solver = pywrapcp.Solver("MineSweeper")

# Feed playground with predefined puzzle
playground = puzzle["1"].copy()

boardSize = len(playground)

# n x n empty Matrix
board = [[None for _j in range(boardSize)] for _i in range(boardSize)]

# Feed board with Minesweeper Puzzle
for row in range(boardSize):
    for column in range(boardSize):
        if(playground[row][column] == -1):
            board[row][column] = (-1, solver.IntVar(-1, 0))
        else:
            board[row][column] = (playground[row][column], solver.IntVar(0, 8))
            solver.Add(board[row][column][1] == board[row][column][0])
            
# --------------------------------------------------------------------------------
# Set constraints
# --------------------------------------------------------------------------------
for row in range(boardSize):
    for column in range(boardSize):
        if (board[row][column][0] != -1):
            rowRange = 0
            columnRange = 0
            rowOffset = 0
            columnOffset = 0
            
            if (row == 0):
                if (column == 0):
                    rowRange = 2
                    columnRange = 2
                elif (column == boardSize - 1):
                    rowRange = 2
                    columnRange = 2
                    rowOffset = 0
                    columnOffset = -1
                else:
                    rowRange = 2
                    columnRange = 3
                    rowOffset = 0
                    columnOffset = -1                
            
            elif (row == boardSize - 1):
                if (column == 0):
                    rowRange = 2
                    columnRange = 2
                    rowOffset = -1
                elif (column == boardSize - 1):
                    rowRange = 2
                    columnRange = 2
                    rowOffset = -1
                    columnOffset = -1
                else:
                    rowRange = 2
                    columnRange = 3
                    rowOffset = -1
                    columnOffset = -1
            
            else:
                if (column == 0):
                    rowRange = 3
                    columnRange = 2
                    rowOffset = -1
                elif (column == boardSize - 1):
                    rowRange = 3
                    columnRange = 2
                    rowOffset = -1
                    columnOffset = -1
                else:
                    rowRange = 3
                    columnRange = 3
                    rowOffset = -1
                    columnOffset = -1  
            
            # The trick here is to tell the solver, that the sum of all "-1"-values around the given number should be
            # equal the negative value of the given number.
            solver.Add(solver.Sum(
                    [tuple[1] for tuple in 
                       [board[row + rowOffset + i][column + columnOffset + j] for i in range(rowRange) for j in range(columnRange)]
                       if tuple[0] == -1]) == -board[row][column][0])
                
# --------------------------------------------------------------------------------
# Now solve it!
# --------------------------------------------------------------------------------
db = solver.Phase(list(board[i][j][1] for i in range(boardSize) for j in range(boardSize)), # Sudoku board as array
                solver.INT_VAR_SIMPLE,         # Variable selection policy for search
                solver.INT_VALUE_SIMPLE)       # Value selection policy for search

solver.NewSearch(db)

if solver.NextSolution():
    printSolution(board)
    
solver.EndSearch()

#Display the number of solutions found:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime: {}ms".format(solver.WallTime()))
print("Failures: {}".format(solver.Failures()))
print("Branches: {}".format(solver.Branches()))

Possible solution:

1  ∙  ∙  ●  2  ∙  ●  1  
●  1  2  ●  3  ∙  ∙  1  
∙  2  ∙  1  ∙  ●  ∙  0  
●  2  1  ∙  ∙  2  3  1  
∙  ∙  ●  2  ∙  ●  ∙  ●  
1  ∙  ∙  ●  4  3  ●  ∙  
●  1  ∙  ●  4  ●  3  ●  
1  ∙  ∙  2  ●  2  ∙  1  

Solutions: 1
Runtime: 15ms
Failures: 4
Branches: 8
